# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nhl_FNames = pd.Series({'Rangers': 'New York Rangers', 
                        'Islanders': 'New York Islanders', 
                        'Devils': 'New Jersey Devils', 
                        'Kings': 'Los Angeles Kings', 
                        'Ducks': 'Anaheim Ducks', 
                        'Sharks': 'San Jose Sharks',
                        'Blackhawks': 'Chicago Blackhawks', 
                        'Stars': 'Dallas Stars', 
                        'Capitals': 'Washington Capitals',
                        'Flyers': 'Philadelphia Flyers', 
                        'Bruins': 'Boston Bruins', 
                        'Wild': 'Minnesota Wild',
                        'Avalanche': 'Colorado Avalanche', 
                        'Panthers':  'Florida Panthers',
                        'Coyotes': 'Arizona Coyotes',
                        'Red': 'Detroit Red Wings',
                        'Maple': 'Toronto Maple Leafs',
                        'Lightning': 'Tampa Bay Lightning',
                        'Penguins': 'Pittsburgh Penguins',
                        'Blues': 'St. Louis Blues',
                        'Predators': 'Nashville Predators',
                        'Sabres': 'Buffalo Sabres',
                        'Canadiens': 'Montreal Canadiens',
                        'Canucks': 'Vancouver Canucks',
                        'Blue': 'Columbus Blue Jackets',
                        'Flames': 'Calgary Flames',
                        'Senators': 'Ottawa Senators',
                        'Oilers': 'Edmonton Oilers',
                        'Jets': 'Winnipeg Jets',
                        'Golden': 'Vegas Golden Knights',
                        'Hurricanes': 'Carolina Hurricanes'})

def handleNHLName(name):
    try:
        return nhl_FNames.loc[name]
    except KeyError:
        return np.NaN

# Ajust the names of the values for the question
nhl_df = nhl_df.where(nhl_df['year'].eq(2018)).dropna()
nhl_df = nhl_df[['team', 'W', 'L']].where(nhl_df['W'].str.isnumeric()).dropna()
nhl_df['team'] = nhl_df['team'].replace('\*$', '', regex=True)

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'NHL']]
cities['NHL'] = cities['NHL'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities.columns = ['Metropolitan area', 'Population','team']
cities['team'] = cities['team'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x)).drop(columns=[0,2, 4, 6], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities = cities.explode('team')
cities['team'] = cities['team'].apply(lambda x: handleNHLName(x))
cities = cities.dropna().merge(nhl_df, how='left', on='team')
cities['WinLossRatio'] = cities['W'].astype('int32') / (cities['W'].astype('int32') + cities['L'].astype('int32'))

def nhl_correlation(): 
    answer = cities[['Metropolitan area', 'Population', 'WinLossRatio']].groupby(['Metropolitan area', 'Population'], sort=False).agg({'WinLossRatio': 'mean'})
    answer.reset_index(inplace=True)
    #print(answer['Population'].dtype)
    #print(answer['WinLossRatio'].dtype)
    
    population_by_region = answer['Population'].astype('float64') # pass in metropolitan area population from cities
    win_loss_by_region = answer['WinLossRatio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nba_FNames = pd.Series({'Knicks': 'New York Knicks',
                        'Nets': 'Brooklyn Nets',
                        'Lakers': 'Los Angeles Lakers',
                        'Clippers': 'Los Angeles Clippers',
                        'Warriors': 'Golden State Warriors',
                        'Bulls': 'Chicago Bulls',
                        'Mavericks': 'Dallas Mavericks',
                        'Wizards': 'Washington Wizards',
                        '76ers': 'Philadelphia 76ers',
                        'Celtics': 'Boston Celtics',
                        'Timberwolves': 'Minnesota Timberwolves',
                        'Nuggets': 'Denver Nuggets',
                        'Heat': 'Miami Heat',
                        'Suns': 'Phoenix Suns',
                        'Pistons': 'Detroit Pistons',
                        'Raptors': 'Toronto Raptors',
                        'Rockets': 'Houston Rockets',
                        'Hawks': 'Atlanta Hawks',
                        'Cavaliers': 'Cleveland Cavaliers',
                        'Hornets': 'Charlotte Hornets',
                        'Pacers': 'Indiana Pacers',
                        'Bucks': 'Milwaukee Bucks',
                        'Pelicans': 'New Orleans Pelicans',
                        'Magic': 'Orlando Magic',
                        'Trail': 'Portland Trail Blazers',
                        'Jazz': 'Utah Jazz',
                        'Spurs': 'San Antonio Spurs',
                        'Kings': 'Sacramento Kings',
                        'Thunder': 'Oklahoma City Thunder',
                        'Grizzlies': 'Memphis Grizzlies'})
def handleNBANames(name):
    try:
        return nba_FNames.loc[name]
    except KeyError:
        return np.NaN

# Ajust the names of the values for the question
nba_df = nba_df.where(nba_df['year'].eq(2018)).dropna()
nba_df = nba_df[['team', 'W', 'L']].where(nba_df['W'].str.isnumeric()).dropna()
nba_df['team'] = nba_df['team'].replace('\((\d+)\)$', '', regex=True).replace('\*\xa0$', '', regex=True).replace('\xa0$', '', regex=True)

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'NBA']]
cities['NBA'] = cities['NBA'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities.columns = ['Metropolitan area', 'Population','team']
teams = cities['team'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x))
teams.iloc[6].at[1] = teams.iloc[6].at[0]
teams = teams.drop(columns=[0, 2, 4], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities['team'] = teams
cities = cities.explode('team')
cities['team'] = cities['team'].apply(lambda x: handleNBANames(x))
cities = cities.dropna().merge(nba_df, how='left', on='team')
cities['WinLossRatio'] = cities['W'].astype('int32') / (cities['W'].astype('int32') + cities['L'].astype('int32'))

def nba_correlation():
    answer = cities[['Metropolitan area', 'Population', 'WinLossRatio']].groupby(['Metropolitan area', 'Population'], sort=False).agg({'WinLossRatio': 'mean'})
    answer.reset_index(inplace=True)
    #print(answer['Population'].dtype)
    #print(answer['WinLossRatio'].dtype)
    
    population_by_region = answer['Population'].astype('float64') # pass in metropolitan area population from cities
    win_loss_by_region = answer['WinLossRatio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return corr

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

mlb_FNames = pd.Series({'Yankees': 'New York Yankees',
                        'Mets': 'New York Mets',
                        'Dodgers': 'Los Angeles Dodgers',
                        'Angels': 'Los Angeles Angels',
                        'Giants': 'San Francisco Giants',
                        'Athletics': 'Oakland Athletics',
                        'Cubs' : 'Chicago Cubs',
                        'White': 'Chicago White Sox',
                        'Rangers': 'Texas Rangers',
                        'Nationals': 'Washington Nationals',
                        'Phillies': 'Philadelphia Phillies',
                        'Red': 'Boston Red Sox',
                        'Twins': 'Minnesota Twins',
                        'Rockies': 'Colorado Rockies',
                        'Marlins': 'Miami Marlins',
                        'Diamondbacks': 'Arizona Diamondbacks',
                        'Tigers': 'Detroit Tigers',
                        'Blue': 'Toronto Blue Jays',
                        'Astros': 'Houston Astros',
                        'Braves': 'Atlanta Braves',
                        'Rays': 'Tampa Bay Rays',
                        'Pirates': 'Pittsburgh Pirates',
                        'Indians': 'Cleveland Indians',
                        'Mariners': 'Seattle Mariners',
                        'Reds': 'Cincinnati Reds',
                        'Royals': 'Kansas City Royals',
                        'Cardinals': 'St. Louis Cardinals',
                        'Orioles': 'Baltimore Orioles',
                        'Brewers': 'Milwaukee Brewers',
                        'Padres': 'San Diego Padres'})

def handleMLBNames(name):
    try:
        return mlb_FNames.loc[name]
    except KeyError:
        return np.NaN

# Adjust the names of the values for the question
mlb_df = mlb_df.where(mlb_df['year'].eq(2018)).dropna()
mlb_df = mlb_df[['team', 'W', 'L']]

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'MLB']]
cities['MLB'] = cities['MLB'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities.columns = ['Metropolitan area', 'Population','team']
cities['team'] = cities['team'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x)).drop(columns=[0, 2, 4, 6], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities = cities.explode('team')
cities['team'] = cities['team'].apply(lambda x: handleMLBNames(x))
cities = cities.dropna().merge(mlb_df, how='left', on='team')
cities['WinLossRatio'] = cities['W'].astype('int32') / (cities['W'].astype('int32') + cities['L'].astype('int32'))

def mlb_correlation(): 
    answer = cities[['Metropolitan area', 'Population', 'WinLossRatio']].groupby(['Metropolitan area', 'Population'], sort=False).agg({'WinLossRatio': 'mean'})
    answer.reset_index(inplace=True)
    #print(answer['Population'].dtype)
    #print(answer['WinLossRatio'].dtype)
    
    population_by_region = answer['Population'].astype('float64') # pass in metropolitan area population from cities
    win_loss_by_region = answer['WinLossRatio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return corr

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nfl_FNames = pd.Series({'Giants': 'New York Giants',
                         'Jets': 'New York Jets',
                         'Rams': 'Los Angeles Rams',
                         'Chargers': 'Los Angeles Chargers',
                         '49ers': 'San Francisco 49ers',
                         'Raiders': 'Oakland Raiders',
                         'Bears': 'Chicago Bears',
                         'Cowboys': 'Dallas Cowboys',
                         'Redskins': 'Washington Redskins',
                         'Eagles': 'Philadelphia Eagles',
                         'Patriots': 'New England Patriots',
                         'Vikings': 'Minnesota Vikings',
                         'Broncos': 'Denver Broncos',
                         'Dolphins': 'Miami Dolphins',
                         'Cardinals': 'Arizona Cardinals',
                         'Lions': 'Detroit Lions',
                         'Texans': 'Houston Texans',
                         'Falcons': 'Atlanta Falcons',
                         'Buccaneers': 'Tampa Bay Buccaneers',
                         'Steelers': 'Pittsburgh Steelers',
                         'Browns': 'Cleveland Browns',
                         'Seahawks': 'Seattle Seahawks',
                         'Bengals': 'Cincinnati Bengals',
                         'Chiefs': 'Kansas City Chiefs',
                         'Ravens': 'Baltimore Ravens',
                         'Panthers': 'Carolina Panthers',
                         'Colts': 'Indianapolis Colts',
                         'Titans': 'Tennessee Titans',
                         'Saints': 'New Orleans Saints',
                         'Bills': 'Buffalo Bills',
                         'Jaguars': 'Jacksonville Jaguars',
                         'Packers': 'Green Bay Packers'})

def handleNFLNames(name):
    try:
        return nfl_FNames.loc[name]
    except KeyError:
        return np.NaN

# Adjust the names of the values for the question
nfl_df = nfl_df.where(nfl_df['year'].eq(2018)).dropna()
nfl_df = nfl_df[['team', 'W', 'L']].where(nfl_df['W'].str.isnumeric()).dropna()
nfl_df['team'] = nfl_df['team'].replace('[\*+]$', '', regex=True)

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'NFL']]
cities['NFL'] = cities['NFL'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities.columns = ['Metropolitan area', 'Population','team']
teams = cities['team'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x))
teams.iloc[2].at[3] = teams.iloc[2].at[1]
teams.iloc[2].at[1] = teams.iloc[2].at[0]
teams = teams.drop(columns=[0, 2, 4], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities['team'] = teams
cities = cities.explode('team')
cities['team'] = cities['team'].apply(lambda x: handleNFLNames(x))
cities = cities.dropna().merge(nfl_df, how='left', on='team')
cities['WinLossRatio'] = cities['W'].astype('int32') / (cities['W'].astype('int32') + cities['L'].astype('int32'))


def nfl_correlation(): 
    answer = cities[['Metropolitan area', 'Population', 'WinLossRatio']].groupby(['Metropolitan area', 'Population'], sort=False).agg({'WinLossRatio': 'mean'})
    answer.reset_index(inplace=True)
    #print(answer['Population'].dtype)
    #print(answer['WinLossRatio'].dtype)
    
    population_by_region = answer['Population'].astype('float64') # pass in metropolitan area population from cities
    win_loss_by_region = answer['WinLossRatio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return corr

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# -------------------NHL 
nhl_df = nhl_df.where(nhl_df['year'].eq(2018)).dropna()
nhl_df = nhl_df[['team', 'W', 'L']].where(nhl_df['W'].str.isnumeric()).dropna()
nhl_df.set_axis(['NHL', 'W', 'L'], axis=1, inplace =True)
nhl_df['NHL'] = nhl_df['NHL'].replace('\*$', '', regex=True)

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'NHL']]
cities['NHL'] = cities['NHL'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities['NHL'] = cities['NHL'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x)).drop(columns=[0,2, 4, 6], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities = cities.explode('NHL')
cities['NHL'] = cities['NHL'].apply(lambda x: handleNHLName(x))
nhl_perf = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
nhl_perf = nhl_perf.dropna().merge(nhl_df, how='left', on='NHL')
nhl_perf['WinLossRatioNHL'] = nhl_perf['W'].astype('int32') / (nhl_perf['W'].astype('int32') + nhl_perf['L'].astype('int32'))
nhl_perf = nhl_perf[['Metropolitan area', 'WinLossRatioNHL']].groupby(['Metropolitan area'], sort=False).agg({'WinLossRatioNHL': 'mean'})
#answer.reset_index(inplace=True)
# -------------------NBA
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# Ajust the names of the values for the question
nba_df = nba_df.where(nba_df['year'].eq(2018)).dropna()
nba_df = nba_df[['team', 'W', 'L']].where(nba_df['W'].str.isnumeric()).dropna()
nba_df.set_axis(['NBA', 'W', 'L'], axis=1, inplace =True)
nba_df['NBA'] = nba_df['NBA'].replace('\((\d+)\)$', '', regex=True).replace('\*\xa0$', '', regex=True).replace('\xa0$', '', regex=True)

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'NBA']]
cities['NBA'] = cities['NBA'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities.columns = ['Metropolitan area', 'Population', 'NBA']
teams = cities['NBA'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x))
teams.iloc[6].at[1] = teams.iloc[6].at[0]
teams = teams.drop(columns=[0, 2, 4], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities['NBA'] = teams
cities = cities.explode('NBA')
cities['NBA'] = cities['NBA'].apply(lambda x: handleNBANames(x))
cities = cities.dropna().merge(nba_df, how='left', on='NBA')
cities['WinLossRatioNBA'] = cities['W'].astype('int32') / (cities['W'].astype('int32') + cities['L'].astype('int32'))
nba_perf = cities[['Metropolitan area', 'WinLossRatioNBA']].groupby(['Metropolitan area'], sort=False).agg({'WinLossRatioNBA': 'mean'})
nba_perf.reset_index(inplace=True)

#------------------MLB
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# Ajust the names of the values for the question
mlb_df = mlb_df.where(mlb_df['year'].eq(2018)).dropna()
mlb_df = mlb_df[['team', 'W', 'L']]
mlb_df.set_axis(['MLB', 'W', 'L'], axis=1, inplace =True)

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'MLB']]
cities['MLB'] = cities['MLB'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities.columns = ['Metropolitan area', 'Population','MLB']
cities['MLB'] = cities['MLB'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x)).drop(columns=[0, 2, 4, 6], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities = cities.explode('MLB')
cities['MLB'] = cities['MLB'].apply(lambda x: handleMLBNames(x))
cities = cities.dropna().merge(mlb_df, how='left', on='MLB')
cities['WinLossRatioMLB'] = cities['W'].astype('int32') / (cities['W'].astype('int32') + cities['L'].astype('int32'))

mlb_perf = cities[['Metropolitan area', 'WinLossRatioMLB']].groupby(['Metropolitan area'], sort=False).agg({'WinLossRatioMLB': 'mean'})
mlb_perf.reset_index(inplace=True)

#---------------------------NFL
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# Ajust the names of the values for the question
nfl_df = nfl_df.where(nfl_df['year'].eq(2018)).dropna()
nfl_df = nfl_df[['team', 'W', 'L']].where(nfl_df['W'].str.isnumeric()).dropna()
nfl_df['team'] = nfl_df['team'].replace('[\*+]$', '', regex=True)
nfl_df.set_axis(['NFL', 'W', 'L'], axis=1, inplace =True)

cities = cities[['Metropolitan area', 'Population (2016 est.)[8]',  'NFL']]
cities['NFL'] = cities['NFL'].replace('[\[].*$', '', regex=True).replace('—', np.NaN).replace('^$', np.NaN, regex=True)
cities.columns = ['Metropolitan area', 'Population','NFL']
teams = cities['NFL'].str.split(r'([A-Z]{1}[a-z]*)').apply(lambda x:pd.Series(x))
teams.iloc[2].at[3] = teams.iloc[2].at[1]
teams.iloc[2].at[1] = teams.iloc[2].at[0]
teams = teams.drop(columns=[0, 2, 4], axis=1).apply(lambda x:x.dropna().to_list(), axis=1)
cities['NFL'] = teams
cities = cities.explode('NFL')
cities['NFL'] = cities['NFL'].apply(lambda x: handleNFLNames(x))
cities = cities.dropna().merge(nfl_df, how='left', on='NFL')
cities['WinLossRatioNFL'] = cities['W'].astype('int32') / (cities['W'].astype('int32') + cities['L'].astype('int32'))

nfl_perf = cities[['Metropolitan area', 'WinLossRatioNFL']].groupby(['Metropolitan area'], sort=False).agg({'WinLossRatioNFL': 'mean'})
nfl_perf.reset_index(inplace=True)

# Merging for all Performaces
performace_df = nhl_perf.merge(nba_perf, how='outer', on='Metropolitan area')
performace_df = performace_df.merge(mlb_perf, how='outer', on='Metropolitan area')
performace_df = performace_df.merge(nfl_perf, how='outer', on='Metropolitan area')
performace_df.set_axis(['Metropolitan area', 'NHL', 'NBA', 'MLB', 'NFL'], axis=1, inplace=True)
performace_df.set_index('Metropolitan area', drop=True, inplace=True)

def sports_team_performance():
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for sport1 in performace_df.columns:
        for sport2 in performace_df.columns:
            if(sport1 != sport2):
                compare = performace_df[[sport1, sport2]].dropna()
                stat, p_value = stats.ttest_rel(compare[sport1], compare[sport2])
                p_values.loc[sport1].at[sport2] = p_value
       
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values